# BioMedBERT - BREATHE using BERT Weights and Vocab

In [1]:
# !sudo pip install tensorflow==1.15

In [1]:
import os
import sys
import json
import tensorflow as tf

In [2]:
tf.__version__

'1.15.0'

Save model assets and checkpoints to GCS

In [3]:
BUCKET_NAME = "ekaba-assets"
MODEL_DIR = "biomedbert_base_bert_weights_and_vocab"
tf.io.gfile.mkdir(MODEL_DIR)

Hyparameter configuration for BERT BASE

In [4]:
# VOC_SIZE = 32000
# VOC_FNAME = "biomedbert-8M.txt"

In [5]:
# # use this for BERT-base

# bert_base_config = {
#     "attention_probs_dropout_prob": 0.1,
#     "directionality": "bidi",
#     "hidden_act": "gelu",
#     "hidden_dropout_prob": 0.1,
#     "hidden_size": 768,
#     "initializer_range": 0.02,
#     "intermediate_size": 3072,
#     "max_position_embeddings": 512,
#     "num_attention_heads": 12,
#     "num_hidden_layers": 12,
#     "pooler_fc_size": 768,
#     "pooler_num_attention_heads": 12,
#     "pooler_num_fc_layers": 3,
#     "pooler_size_per_head": 128,
#     "pooler_type": "first_token_transform",
#     "type_vocab_size": 2,
#     "vocab_size": VOC_SIZE
# }

# with open("{}/bert_config.json".format(MODEL_DIR), "w") as fo:
#     json.dump(bert_base_config, fo, indent=2)

In [6]:
# update vocab
# !cp ../vocabulary/full_text/biomedbert-8M.txt \
# biomedbert_base_bert_weights_breathe_vocab/biomedbert-8M.txt

In [7]:
# %%bash
# cd biomedbert_base_bert_weights_and_vocab
# wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_cased_L-24_H-1024_A-16.zip

In [8]:
# %%bash
# cd biomedbert_base_bert_weights_and_vocab
# unzip wwm_cased_L-24_H-1024_A-16.zip

In [9]:
# %%bash
# mv biomedbert_base_bert_weights_and_vocab/wwm_cased_L-24_H-1024_A-16/* biomedbert_base_bert_weights_and_vocab

In [10]:
# !rm -rf biomedbert_base_bert_weights_and_vocab/wwm_cased_L-24_H-1024_A-16.zip

In [14]:
# !du biomedbert_base_bert_weights_and_vocab/* -h

4.0K	biomedbert_base_bert_weights_and_vocab/bert_config.json
1.3G	biomedbert_base_bert_weights_and_vocab/bert_model.ckpt.data-00000-of-00001
20K	biomedbert_base_bert_weights_and_vocab/bert_model.ckpt.index
1.8M	biomedbert_base_bert_weights_and_vocab/bert_model.ckpt.meta
212K	biomedbert_base_bert_weights_and_vocab/vocab.txt
4.0K	biomedbert_base_bert_weights_and_vocab/wwm_cased_L-24_H-1024_A-16


In [11]:
# # move to GCS
# !gsutil -m cp -r $MODEL_DIR gs://ekaba-assets/

In [14]:
# tf.io.gfile.glob('gs://ekaba-assets/pre_trained_data_full_biomed/*')

In [12]:
init_checkpoint = tf.train.latest_checkpoint('gs://ekaba-assets/biomedbert_base_bert_weights_and_vocab')

In [13]:
init_checkpoint

'gs://ekaba-assets/biomedbert_base_bert_weights_and_vocab/model.ckpt-68000'

In [17]:
%%bash
export INPUT_FILE=gs://ekaba-assets/pre_trained_breathe_bert_vocab/shard_*
export OUTPUT_DIR=gs://ekaba-assets/biomedbert_base_bert_weights_and_vocab
export BERT_BASE_DIR=gs://ekaba-assets/biomedbert_base_bert_weights_and_vocab

python3 bert/run_pretraining.py \
  --input_file="$INPUT_FILE" \
  --output_dir="$OUTPUT_DIR" \
  --do_train=True \
  --do_eval=True \
  --bert_config_file="$BERT_BASE_DIR"/bert_config.json \
  --init_checkpoint="$BERT_BASE_DIR"/bert_model.ckpt \
  --train_batch_size=256 \
  --eval_batch_size=256 \
  --max_seq_length=128 \
  --max_predictions_per_seq=20 \
  --num_train_steps=10000000 \
  --num_warmup_steps=1000 \
  --learning_rate=1e-5 \
  --use_tpu=true \
  --tpu_name=for-shweta-tpu \
  --tpu_zone=europe-west4-a \
  --gcp_project=ai-vs-covid19 \
  --num_tpu_cores=128

Process is interrupted.
